In [24]:
#First mount
import os
print(os.getcwd())

/content


In [25]:
#CNN model building
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


In [26]:
model = Sequential()

#1st layer
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) #Step1 & Step2 - convolution and relu activation function
model.add(MaxPooling2D(pool_size = (2, 2))) #Step3 - MaxPooling

#2nd layer
model.add(Conv2D(32, (3, 3), activation = 'relu')) #Step1 & Step2 - convolution and relu activation function
model.add(MaxPooling2D(pool_size = (2, 2))) #Step3 - MaxPooling

model.add(Flatten()) #Step4 - Flattening

model.add(Dense(units = 128, activation = 'relu')) #64p + 64p = 128p => 128 total input nodes
model.add(Dense(units = 3, activation = 'softmax')) #depth - 3 => output node - 3

In [27]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) #compiling CNN with adam optimizer

In [28]:
model.summary() #Summary of our model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                

In [39]:
#Fitting our CNN model to the images
from keras.preprocessing.image import ImageDataGenerator

train_data_generate = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_data_generate = ImageDataGenerator(rescale = 1./255)

#now we have to add the shortcut to the Drive of our Data folder in My Drive then use below one: importing dataset folders
train_set = train_data_generate.flow_from_directory('/content/drive/MyDrive/Forsk coding school code practices/Image Datasets/training_set', target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

test_set = test_data_generate.flow_from_directory('/content/drive/MyDrive/Forsk coding school code practices/Image Datasets/test_set', target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

Found 445 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [40]:
print(type(train_set))
print(type(train_data_generate))
print(type(test_set))
print(type(test_data_generate))

<class 'keras.preprocessing.image.DirectoryIterator'>
<class 'keras.preprocessing.image.ImageDataGenerator'>
<class 'keras.preprocessing.image.DirectoryIterator'>
<class 'keras.preprocessing.image.ImageDataGenerator'>


In [46]:
model.fit_generator(train_set, steps_per_epoch = 10, epochs = 5, validation_data = test_set, validation_steps = 200) #steps per epoch - one batch size will get repeated this much time in one epoch

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
 4/10 [===========>..................] - ETA: 5s - loss: 0.0706 - accuracy: 0.9766

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/10 [==============================] - 22s 2s/step - loss: 0.0643 - accuracy: 0.9779 - val_loss: 0.2257 - val_accuracy: 0.9202
Epoch 2/5
10/10 [==============================] - 11s 1s/step - loss: 0.0803 - accuracy: 0.9656
Epoch 3/5
10/10 [==============================] - 12s 1s/step - loss: 0.1161 - accuracy: 0.9590
Epoch 4/5
10/10 [==============================] - 11s 1s/step - loss: 0.0923 - accuracy: 0.9621
Epoch 5/5
10/10 [==============================] - 11s 1s/step - loss: 0.0706 - accuracy: 0.9653


***Using the Pretrained Model: VGG16***



In [52]:
from tensorflow.keras.applications import VGG16

In [73]:
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (150, 150, 3)) #include_top=False, means ANN part excluded and CNN part included

In [54]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [55]:
from keras import models
from keras import layers
import os
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [56]:
conv_base.trainable = False #do not train the conv-base

In [58]:
base_dir = '/content/drive/MyDrive/Forsk coding school code practices/Image Datasets'
train_dir = os.path.join(base_dir, 'training_set')

test_dir = os.path.join(base_dir, 'test_set')

In [67]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) #rescale factor is feature scalling, convert values in the range of 0-1 before applying any processing

In [68]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='categorical')

Found 445 images belonging to 3 classes.


In [69]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='categorical')

Found 238 images belonging to 3 classes.


In [71]:
model.compile(loss='categorical_crossentropy',
              optimizer = optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [74]:
history = model.fit_generator(train_generator, steps_per_epoch=10,
                              epochs=5,
                              validation_data=test_generator,
                              validation_steps=50)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
10/10 [==============================] - 105s 11s/step - loss: 0.7692 - acc: 0.7550 - val_loss: 0.7500 - val_acc: 0.7437
Epoch 2/5
10/10 [==============================] - 51s 5s/step - loss: 0.6742 - acc: 0.8350
Epoch 3/5
 5/10 [==============>...............] - ETA: 25s - loss: 0.6216 - acc: 0.8500

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/10 [==============================] - 48s 5s/step - loss: 0.6375 - acc: 0.8216
Epoch 4/5
10/10 [==============================] - 51s 5s/step - loss: 0.5919 - acc: 0.8250
Epoch 5/5
10/10 [==============================] - 51s 5s/step - loss: 0.5483 - acc: 0.8500


***Using the pretrained model: Fine tuning the model***

Another widely used technique for model reuse, complementary to feature extraction, is fine-tuning (see figure 5.19). 
Fine-tuning consists of unfreezing a few of the top layers of a frozen model base used for feature extraction, and jointly training both the newly added part of the model (in this case, the fully connected classifier) and these top layers. This is called ***fine-tuning*** because it slightly adjusts the more
abstract representations of the model being reused, in order to make them more relevant for the problem at hand.

Thus the steps for fine-tuning a network are as follows:

1.   Add your custom network on top of an already-trained base network.
2.   Freeze the base network.
3.   Train the part you added.
4.   Unfreeze some layers in the base network.
5.   Jointly train both these layers and the part you added.

You already completed the first three steps when doing feature extraction. Let’s proceed
with step 4: you’ll unfreeze your conv_base and then freeze individual layers
inside it.

*You’ll* fine-tune the last three convolutional layers, which means all layers up to block4_pool should be frozen, and the layers block5_conv1, block5_conv2, and block5_conv3 should be trainable.

In [75]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
    
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [77]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/10 [==============================] - 133s 14s/step - loss: 0.4347 - acc: 0.8750 - val_loss: 0.4260 - val_acc: 0.8571
Epoch 2/5
10/10 [==============================] - 50s 5s/step - loss: 0.4090 - acc: 0.8650
Epoch 3/5
10/10 [==============================] - 51s 5s/step - loss: 0.3699 - acc: 0.9050
Epoch 4/5
10/10 [==============================] - 50s 5s/step - loss: 0.3550 - acc: 0.8900
Epoch 5/5
10/10 [==============================] - 49s 5s/step - loss: 0.3412 - acc: 0.9000


Here’s what you should take away from the exercises in the past two sections:

1. Convnets are the best type of machine-learning models for computer-vision
tasks. It’s possible to train one from scratch even on a very small dataset, with decent results.

2. On a small dataset, overfitting will be the main issue. Data augmentation is a powerful way to fight overfitting when you’re working with image data.

3. It’s easy to reuse an existing convnet on a new dataset via feature extraction. This is a valuable technique for working with small image datasets.

4. As a complement to feature extraction, you can use fine-tuning, which adapts to a new problem some of the representations previously learned by an existing
model. This pushes performance a bit further.

In [82]:
divmod(23, 5) #returns a tuple => (quotient, remainder)

(4, 3)